Mohamed Sohail Rajab

--------------

LINK TO DATASE: Gerry (2023). BIRDS 525 SPECIES- IMAGE CLASSIFICATION. [online] Kaggle.com. Available at: https://www.kaggle.com/datasets/gpiosenka/100-bird-species/data?select=birds.csv

---------------
### Welcome to this exploration of bird species classification using state-of-the-art deep learning techniques. 

----------

The dataset we're looking at, which contains photos of 525 different bird species, is ideal for image classification with TensorFlow for several reasons:

1. **Large and Diverse Dataset**: The dataset contains 84,635 training photos and a total of 90,000 images across training, test, and validation sets. This variety and volume are essential for building robust deep learning models.


2. **Clean and Preprocessed Data**: The dataset has been cleaned to remove duplicates and low-quality photos, allowing the model to be trained on high-quality, unique data. This stage is critical to preventing overfitting and ensuring that the model learns to generalise from a wide range of photos.


4. **Structured Data Organization**: The dataset's structure, with separate subdirectories for each bird species in training, test, and validation sets, aligns well with TensorFlow's ImageDataGenerator.flow_from_directory method. This method is a convenient way to load and preprocess images for training with Keras models in TensorFlow.

5. **Appropriate Image Resolution**: The images are standardized to a resolution of 224x224x3, which is suitable for most convolutional neural network (CNN) architectures. This uniformity in size allows for consistent input to the model without the need for additional resizing during preprocessing.

6. **Suitable for Transfer Learning**: The dataset's characteristics make it ideal for transfer learning using pre-trained models available in TensorFlow (like ResNet, VGG, EfficientNet). These models, pre-trained on large datasets like ImageNet, can be fine-tuned on this bird species dataset, potentially leading to high accuracy due to the similarity in the data (natural images of objects).

7. **Challenging Real-world Scenario**: The dataset reflects a real-world challenge due to the variation in bird species appearance and the imbalance in the representation of male and female species. This aspect makes it a good case for developing robust image classification models.

8. **Potential for High Accuracy**: Given the dataset's quality and diversity, even moderately complex models are expected to achieve high training and test accuracies, as mentioned in the dataset description. This indicates that the dataset is well-prepared for deep learning applications.

9. **Availability of Metadata**: The inclusion of a CSV file with labels, scientific names, and dataset partitions (train, test, validation) provides valuable metadata that can be used for more in-depth analysis and understanding of the model's performance.

In summary, this dataset's size, quality, and organization make it a prime candidate for training effective image classification models using TensorFlow, particularly those employing CNN architectures or transfer learning techniques.

-----------------

- **Exploratory Analysis and Data Wrangling**:  In order to understand the properties of the bird species image dataset, I will first undertake exploratory data analysis (EDA). This includes visualising the distribution of classes (bird species), detecting imbalances in the dataset, and assessing image quality. Data wrangling may entail organising the data into a structure suitable for TensorFlow training, such as organising photographs into folders by class or preprocessing them to a uniform size and format.

- **Preparing the Environment**: The notebook contains cells for installing required programmes such as keras-preprocessing and downloading assistance functions. These steps are essential for getting the Python environment ready for data processing and model training.

- **Configuration Parameters**: It sets certain parameters like BATCH_SIZE and TARGET_SIZE, which are likely used for image processing and feeding into the neural network.

- **Exploratory Analysis and Data Wrangling**: The notebook likely includes steps to explore and preprocess the dataset. This would involve examining the images of birds, understanding their distribution across different classes (species), and preparing the data for model training (resizing images, normalization, etc.).

- **Information Conveyed by Tables or Graphs**: Any tables or graphs in the notebook would likely show the distribution of data (like the number of images per species), model performance metrics (accuracy, loss over epochs), and possibly visualizations of the model's predictions.

- **Dealing with Overfitting and Underfitting**: To address these issues, the notebook might employ techniques like data augmentation (to increase the diversity of the training set), dropout layers in the neural network (to prevent over-reliance on certain neurons), and early stopping (to halt training before the model overfits). 

- **Methods**: We will be making use of nerural networks, more specifically CNNs, we will be using the pre-trained model 'EfficientNetB0'.

- **Hyperparameter Tuning**: This could involve adjusting parameters like learning rate, batch size, number of layers, and neurons in each layer to improve model performance. The notebook might use a systematic approach like grid search or random search to find the best combination of hyperparameters.

- **Interpreting Results**: The analysis of model scores would involve looking at accuracy, precision, recall, and F1 scores. The confusion matrix would provide insights into which species are correctly classified and which are commonly confused. The effects of hyperparameter tuning on these metrics would also be examined to understand the model's performance improvements.



----------------------


In [ ]:
!pip install keras-preprocessing

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow import keras
from tensorflow.keras import layers,models
from keras_preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import Callback, EarlyStopping,ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras import Model
from tensorflow.keras.layers.experimental import preprocessing
from pathlib import Path
import os.path
import random
import matplotlib.cm as cm
import cv2
import seaborn as sns
sns.set_style('darkgrid')
from sklearn.metrics import classification_report, confusion_matrix
import itertools
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings('ignore')


In [ ]:
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
from helper_functions import create_tensorboard_callback, plot_loss_curves, unzip_data, compare_historys, walk_through_dir, pred_and_plot

In [ ]:
BATCH_SIZE = 32
TARGET_SIZE = (224, 224)

### Loading up the dataset

In [ ]:
dataset = "../input/100-bird-species/train"
walk_through_dir(dataset); #to get better understanding of dataset structure 

In [ ]:
image_dir = Path(dataset)

filepaths = list(image_dir.glob(r'**/*.JPG')) + list(image_dir.glob(r'**/*.jpg')) + list(image_dir.glob(r'**/*.png')) + list(image_dir.glob(r'**/*.png'))

labels = list(map(lambda x: os.path.split(os.path.split(x)[0])[1], filepaths))

filepaths = pd.Series(filepaths, name='Filepath').astype(str)
labels = pd.Series(labels, name='Label')

image_df = pd.concat([filepaths, labels], axis=1)

### Visualizations 

In [ ]:
label_counts = image_df['Label'].value_counts()[:20]

plt.figure(figsize=(20, 6))
sns.barplot(x=label_counts.index, y=label_counts.values, alpha=0.8)
plt.title('Distribution of Top 20 Labels in Image Dataset', fontsize=16)
plt.xlabel('Label', fontsize=14)
plt.ylabel('Count', fontsize=14)
plt.xticks(rotation=45)
plt.show()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Load the dataset
birds_df = pd.read_csv('../input/100-bird-species/birds.csv')

# Class Balance in Training Set
plt.figure(figsize=(10, 300))  # Increase the figure size accordingly
sns.countplot(y='labels', data=birds_df[birds_df['data set'] == 'train'], order=birds_df['labels'].value_counts().index)
plt.title('Class Distribution in Training Set')
plt.xlabel('Count')
plt.ylabel('Labels')
plt.show()


In [ ]:
# Comparison of Dataset Splits
dataset_counts = birds_df['data set'].value_counts()
plt.figure(figsize=(10, 6))
sns.barplot(x=dataset_counts.index, y=dataset_counts.values, alpha=0.8)
plt.title('Comparison of Dataset Splits')
plt.ylabel('Number of Images')
plt.show()

In [ ]:
# Word Cloud of Species Names
from wordcloud import WordCloud

# Generating the word cloud data
wordcloud_data = birds_df['labels'].str.cat(sep=' ')
wordcloud = WordCloud(width=800, height=400, background_color ='white').generate(wordcloud_data)

# Displaying the word cloud
plt.figure(figsize=(15, 8))
plt.imshow(wordcloud)
plt.axis('off')
plt.title('Word Cloud of Species Names')
plt.show()

In [ ]:
random_index = np.random.randint(0, len(image_df), 16)
fig, axes = plt.subplots(nrows=4, ncols=4, figsize=(10, 10),
                        subplot_kw={'xticks': [], 'yticks': []})

for i, ax in enumerate(axes.flat):
    ax.imshow(plt.imread(image_df.Filepath[random_index[i]]))
    ax.set_title(image_df.Label[random_index[i]])
plt.tight_layout()
plt.show()

### Pre-processing 

In [ ]:
# masouduut94 (2021). Forgery Detection by using extracted ELA features. 
# [online] Kaggle.com. Available at: https://www.kaggle.com/code/masouduut94/forgery-detection-by-using-extracted-ela-features


def compute_ela_cv(path, quality):
    temp_filename = 'temp_file_name.jpeg'
    SCALE = 15
    orig_img = cv2.imread(path)
    orig_img = cv2.cvtColor(orig_img, cv2.COLOR_BGR2RGB)
    
    cv2.imwrite(temp_filename, orig_img, [cv2.IMWRITE_JPEG_QUALITY, quality])

    compressed_img = cv2.imread(temp_filename)

    diff = SCALE * cv2.absdiff(orig_img, compressed_img)
    return diff


def convert_to_ela_image(path, quality):
    temp_filename = 'temp_file_name.jpeg'
    ela_filename = 'temp_ela.png'
    image = Image.open(path).convert('RGB')
    image.save(temp_filename, 'JPEG', quality = quality)
    temp_image = Image.open(temp_filename)

    ela_image = ImageChops.difference(image, temp_image)

    extrema = ela_image.getextrema()
    max_diff = max([ex[1] for ex in extrema])
    if max_diff == 0:
        max_diff = 1

    scale = 255.0 / max_diff
    ela_image = ImageEnhance.Brightness(ela_image).enhance(scale)
    
    return ela_image


def random_sample(path, extension=None):
    if extension:
        items = Path(path).glob(f'*.{extension}')
    else:
        items = Path(path).glob(f'*')
        
    items = list(items)
        
    p = random.choice(items)
    return p.as_posix()

In [ ]:
p = random_sample('../input/100-bird-species/train/SCARLET MACAW')
orig = cv2.imread(p)
orig = cv2.cvtColor(orig, cv2.COLOR_BGR2RGB) / 255.0
init_val = 100
columns = 3
rows = 3

fig=plt.figure(figsize=(15, 10))
for i in range(1, columns*rows +1):
    quality=init_val - (i-1) * 8
    img = compute_ela_cv(path=p, quality=quality)
    if i == 1:
        img = orig.copy()
    ax = fig.add_subplot(rows, columns, i) 
    ax.title.set_text(f'q: {quality}')
    plt.imshow(img)
plt.show()

In [ ]:
train_df, test_df = train_test_split(image_df, test_size=0.2, shuffle=True, random_state=42)

In [ ]:
train_generator = ImageDataGenerator(
    preprocessing_function=tf.keras.applications.efficientnet.preprocess_input,
    validation_split=0.2
)

test_generator = ImageDataGenerator(
    preprocessing_function=tf.keras.applications.efficientnet.preprocess_input,
)

In [ ]:
train_images = train_generator.flow_from_dataframe(
    dataframe=train_df,
    x_col='Filepath',
    y_col='Label',
    target_size=TARGET_SIZE,
    color_mode='rgb',
    class_mode='categorical',
    batch_size=BATCH_SIZE,
    shuffle=True,
    seed=42,
    subset='training'
)

val_images = train_generator.flow_from_dataframe(
    dataframe=train_df,
    x_col='Filepath',
    y_col='Label',
    target_size=TARGET_SIZE,
    color_mode='rgb',
    class_mode='categorical',
    batch_size=BATCH_SIZE,
    shuffle=True,
    seed=42,
    subset='validation'
)

test_images = test_generator.flow_from_dataframe(
    dataframe=test_df,
    x_col='Filepath',
    y_col='Label',
    target_size=TARGET_SIZE,
    color_mode='rgb',
    class_mode='categorical',
    batch_size=BATCH_SIZE,
    shuffle=False
)

In [ ]:
# Team, K. (2020). Keras documentation: Working with preprocessing layers. 
#[online] Keras.io. Available at: https://keras.io/guides/preprocessing_layers/ 

augment = tf.keras.Sequential([
  layers.experimental.preprocessing.Resizing(224,224),
  layers.experimental.preprocessing.Rescaling(1./255),
  layers.experimental.preprocessing.RandomFlip("horizontal"),
  layers.experimental.preprocessing.RandomRotation(0.1),
  layers.experimental.preprocessing.RandomZoom(0.1),
  layers.experimental.preprocessing.RandomContrast(0.1),
])

In [ ]:
#Team, K. (2020). Keras documentation: Image classification via fine-tuning with EfficientNet. 
#[online] Keras.io. Available at: https://keras.io/examples/vision/image_classification_efficientnet_fine_tuning/
#TRANSFER LEARNING

pretrained_model = tf.keras.applications.efficientnet.EfficientNetB0(
    input_shape=(224, 224, 3),
    include_top=False,
    weights='imagenet',
    pooling='max'
)

pretrained_model.trainable = False



In [ ]:
#Team, K. (2023). Keras documentation: EarlyStopping. 
#[online] Keras.io. Available at: https://keras.io/api/callbacks/early_stopping/

checkpoint_path = "birds_classification_model_checkpoint"
checkpoint_callback = ModelCheckpoint(checkpoint_path,
                                      save_weights_only=True,
                                      monitor="val_accuracy",
                                      save_best_only=True)

early_stopping = EarlyStopping(monitor = "val_loss",
                               patience = 5,
                               restore_best_weights = True) 

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=1e-6)

### CNN and Pipeline 

In [ ]:
#Image classification (2023). Image classification. [online] TensorFlow. Available at: 
#https://www.tensorflow.org/tutorials/images/classification
#PIPELINE
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard, ReduceLROnPlateau
from tensorflow.keras import layers


# Define the input layer
inputs = pretrained_model.input

data_augmentation = tf.keras.Sequential([
    layers.experimental.preprocessing.RandomFlip("horizontal"),
    layers.experimental.preprocessing.RandomRotation(0.1),
    layers.experimental.preprocessing.RandomZoom(0.2),
])

x = data_augmentation(inputs)


x = pretrained_model(x, training=True)

x = Flatten()(x) 
x = Dense(128, activation='relu')(x)
x = Dropout(0.45)(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.45)(x)

# Output layer
outputs = Dense(525, activation='softmax')(x)

model = Model(inputs=inputs, outputs=outputs)

# Compile 
model.compile(
    optimizer=Adam(0.0001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)


early_stopping = EarlyStopping(monitor='val_loss', patience=5)
checkpoint_callback = ModelCheckpoint('model_checkpoint.h5', save_best_only=True)
tensorboard_callback = TensorBoard(log_dir='training_logs')
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2)

# Train the model
history = model.fit(
    train_images,  
    steps_per_epoch=len(train_images),
    validation_data=val_images,  
    validation_steps=len(val_images),
    epochs=80,
    callbacks=[
        early_stopping,
        tensorboard_callback,
        checkpoint_callback,
        reduce_lr
    ]
)


### Model Evaluation 

In [ ]:
results = model.evaluate(test_images, verbose=0)
print("Test Loss: {:.5f}".format(results[0]))
print("Test Accuracy: {:.2f}%".format(results[1] * 100))

### Visualizing loss

In [ ]:
accuracy = history.history['accuracy']
val_accuracy = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(accuracy))
plt.plot(epochs, accuracy, 'b', label='Training accuracy')
plt.plot(epochs, val_accuracy, 'r', label='Validation accuracy')

plt.title('Training and validation accuracy')
plt.legend()
plt.figure()
plt.plot(epochs, loss, 'b', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')

plt.title('Training and validation loss')
plt.legend()
plt.show()

### Making predictions on the Test Data

In [ ]:
pred = model.predict(test_images)
pred = np.argmax(pred,axis=1)

labels = (train_images.class_indices)
labels = dict((v,k) for k,v in labels.items())
pred = [labels[k] for k in pred]

print(f'The first 5 predictions: {pred[:5]}')

### PREDICTION TIME !!!

In [ ]:
random_index = np.random.randint(0, len(test_df) - 1, 15)
fig, axes = plt.subplots(nrows=3, ncols=5, figsize=(25, 15),
                        subplot_kw={'xticks': [], 'yticks': []})

for i, ax in enumerate(axes.flat):
    ax.imshow(plt.imread(test_df.Filepath.iloc[random_index[i]]))
    if test_df.Label.iloc[random_index[i]] == pred[random_index[i]]:
        color = "green"
    else:
        color = "red"
    ax.set_title(f"True: {test_df.Label.iloc[random_index[i]]}\nPredicted: {pred[random_index[i]]}", color=color)
plt.show()
plt.tight_layout()

### Plotting the Classification Reports and Confusion Matrix

In [ ]:
y_test = list(test_df.Label)
print(classification_report(y_test, pred))

### Final Model Summary 

In [ ]:
report = classification_report(y_test, pred, output_dict=True)
df = pd.DataFrame(report).transpose()
df

As you can see from the summary above, we are getting really good scores! 

some future improvements would be incresing the epochs testing time, we can increse epochs from 80 to 200 to test for better, more stable, acuraccy.